In [10]:
from src.api import get_league_data
from src.config import config
from src.utils import cache_summary
import tqdm

LEAGUE_ID = config.league_id
YEARS = config.years
cookies = config.cookies
headers = config.headers

MAX_PERIODS = 20  # overshoot safe; ESPN just returns no data

In [12]:
print("\n📊 Cache BEFORE population:")
display(cache_summary(YEARS, max_period=MAX_PERIODS, league_id=LEAGUE_ID))


📊 Cache BEFORE population:


,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
2025,✅,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜
2024,✅,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜
2023,✅,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜
2022,✅,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜
2021,✅,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜
2020,✅,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜
2019,✅,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜
2018,✅,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜
2017,✗,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜
2016,✗,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜,⬜


In [ ]:
from tqdm import tqdm
import logging

# Make tqdm and logging play nice
class TqdmLoggingHandler(logging.StreamHandler):
    def emit(self, record):
        try:
            msg = self.format(record)
            tqdm.write(msg)  # ensures log lines don't overwrite bars
            self.flush()
        except Exception:
            self.handleError(record)

# Attach handler once
root_logger = logging.getLogger()
if not any(isinstance(h, TqdmLoggingHandler) for h in root_logger.handlers):
    handler = TqdmLoggingHandler()
    handler.setFormatter(logging.Formatter("%(levelname)s: %(message)s"))
    root_logger.addHandler(handler)
    root_logger.setLevel(logging.INFO)

# --- Population loop ---
for year in YEARS:
    for period in tqdm(range(1, MAX_PERIODS + 1), desc=f"Year {year}"):
        data = get_league_data(
            LEAGUE_ID,
            year,
            scoring_period=period,
            cookies=cookies,
            headers=headers,
            use_cache=True,       # respect existing cache
            force_refresh=False,  # don’t blow away existing
        )
        if data is None:
            # Skipped: either future week (retry later) or past-year miss (cached as None)
            continue


In [14]:
print("\n📊 Cache AFTER population:")
display(cache_summary(YEARS, max_period=MAX_PERIODS, league_id=LEAGUE_ID))


📊 Cache AFTER population:


,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
2025,✅,✅,✅,✅,✅,✅,✅,✅,✅,✅,✅,✅,✅,✅,⬜,⬜,⬜,⬜,⬜,⬜
2024,✅,✅,✅,✅,✅,✅,✅,✅,✅,✅,✅,✅,✅,✅,✅,✅,✅,✗,✗,✗
2023,✅,✅,✅,✅,✅,✅,✅,✅,✅,✅,✅,✅,✅,✅,✅,✅,✅,✗,✗,✗
2022,✅,✅,✅,✅,✅,✅,✅,✅,✅,✅,✅,✅,✅,✅,✅,✅,✅,✗,✗,✗
2021,✅,✅,✅,✅,✅,✅,✅,✅,✅,✅,✅,✅,✅,✅,✅,✅,✅,✗,✗,✗
2020,✅,✅,✅,✅,✅,✅,✅,✅,✅,✅,✅,✅,✅,✅,✅,✅,✗,✗,✗,✗
2019,✅,✅,✅,✅,✅,✅,✅,✅,✅,✅,✅,✅,✅,✅,✅,✅,✗,✗,✗,✗
2018,✅,✅,✅,✅,✅,✅,✅,✅,✅,✅,✅,✅,✅,✅,✅,✅,✗,✗,✗,✗
2017,✗,✗,✗,✗,✗,✗,✗,✗,✗,✗,✗,✗,✗,✗,✗,✗,✗,✗,✗,✗
2016,✗,✗,✗,✗,✗,✗,✗,✗,✗,✗,✗,✗,✗,✗,✗,✗,✗,✗,✗,✗


In [21]:
from src.data import load_cache_entry
import json

def inspect_matchups(league_id, year, period, views=("mMatchup", "mTeam")):
    """
    Inspect cached matchups for a given year + scoring period.
    """
    data = load_cache_entry(league_id, year, period, list(views))
    if data is None:
        print(f"❌ No cache entry for {year} period {period}")
        return

    # Filter to actual matchups for this scoring period
    matches = [
        row for row in data.get("schedule", [])
        if row.get("matchupPeriodId") == period
    ]

    print(f"✅ Cache entry loaded for {year} period {period}")
    print("Teams length:", len(data.get("teams", [])))
    print(f"Matchups found for period {period}: {len(matches)}")

    if matches:
        for m in matches:
            home = m["home"]["teamId"]
            away = m["away"]["teamId"]
            home_points = m["home"].get("totalPoints", 0)
            away_points = m["away"].get("totalPoints", 0)
            print(f"Week {period}: Team {home} ({home_points}) vs Team {away} ({away_points})")
    else:
        print("⚠️ No matchups in schedule for this period")

# ---- Example usage ----
inspect_matchups(LEAGUE_ID, 2023, 10)


✅ Cache entry loaded for 2023 period 10
Teams length: 12
Matchups found for period 10: 6
Week 10: Team 8 (111.0) vs Team 1 (150.1)
Week 10: Team 2 (117.2) vs Team 4 (129.0)
Week 10: Team 5 (88.7) vs Team 6 (103.4)
Week 10: Team 11 (91.1) vs Team 12 (116.8)
Week 10: Team 7 (136.5) vs Team 9 (119.4)
Week 10: Team 10 (97.8) vs Team 3 (75.6)
